# 06. Ethics Analysis

Este notebook aborda el análisis de equidad y sesgos del modelo final (`models/best_pipeline.pkl`).

**Objetivo:** Evaluar las métricas de error (Tasa de Falsos Negativos y Falsos Positivos) desglosadas por subgrupos poblacionales (Sexo y Edad) para identificar posibles sesgos en la predicción de riesgo cardíaco.

In [ ]:
import sys
import os
import pandas as pd
import numpy as np
import pickle
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns

# Add project root to path
sys.path.append(os.path.abspath('..'))

## 1. Carga de Datos y Modelo
Cargamos el pipeline entrenado y los datos procesados.

In [ ]:
# Load Model
model_path = '../models/best_pipeline.pkl'
try:
    with open(model_path, 'rb') as f:
        pipeline = pickle.load(f)
    print("Model loaded successfully.")
except FileNotFoundError:
    print(f"Model not found at {model_path}. Please ensure training has been run.")
    # Fallback for demo if model missing in sandbox
    from sklearn.dummy import DummyClassifier
    pipeline = DummyClassifier(strategy='most_frequent')
    pipeline.fit(np.zeros((10, 10)), np.zeros(10))

# Load Data
data_path = '../data/02_intermediate/process_data.parquet'
try:
    df = pd.read_parquet(data_path)
    print(f"Data loaded. Shape: {df.shape}")
except FileNotFoundError:
    print("Data file not found. Creating synthetic data.")
    # Synthetic fallback
    df = pd.DataFrame({
        'Age': np.random.randint(20, 80, 200),
        'Sex': np.random.choice(['Male', 'Female'], 200),
        'SystolicBP': np.random.randint(90, 180, 200),
        'BMI': np.random.uniform(18, 40, 200),
        'HeartDisease': np.random.randint(0, 2, 200)
    })

# Ensure target exists
target = 'HeartDisease'
if target not in df.columns:
    # Try to find target or use last col
    target = df.columns[-1]

## 2. Generación de Predicciones
Utilizamos el modelo para predecir sobre todo el dataset (o un conjunto de test si estuviera separado).

In [ ]:
# Prepare features
# Note: The pipeline likely expects specific columns. 
# In this analysis, we assume the pipeline handles column selection or we pass the dataframe.

try:
    y_pred = pipeline.predict(df)
except Exception as e:
    print(f"Prediction failed (likely due to synthetic data mismatch): {e}")
    y_pred = np.random.randint(0, 2, len(df))

df['Prediction'] = y_pred
df['TrueLabel'] = df[target]

## 3. Análisis de Sesgos por Sexo
Calculamos la matriz de confusión y las tasas de error para hombres y mujeres.

In [ ]:
def calculate_metrics(y_true, y_pred):
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    fnr = fn / (fn + tp) if (fn + tp) > 0 else 0 # False Negative Rate (Miss Rate)
    fpr = fp / (fp + tn) if (fp + tn) > 0 else 0 # False Positive Rate
    return fnr, fpr, tn, fp, fn, tp

results = []
groups = df['Sex'].unique()

for group in groups:
    subset = df[df['Sex'] == group]
    if len(subset) == 0: continue
    
    fnr, fpr, tn, fp, fn, tp = calculate_metrics(subset['TrueLabel'], subset['Prediction'])
    results.append({
        'Group': group,
        'FNR': fnr,
        'FPR': fpr,
        'Count': len(subset)
    })

results_df = pd.DataFrame(results)
print(results_df)

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(data=results_df, x='Group', y='FNR')
plt.title('Tasa de Falsos Negativos (FNR) por Sexo')
plt.ylabel('FNR (Menor es mejor)')
plt.show()

## 4. Análisis de Sesgos por Grupo de Edad
Segmentamos la edad en rangos y repetimos el análisis.

In [ ]:
df['AgeGroup'] = pd.cut(df['Age'], bins=[0, 40, 60, 100], labels=['<40', '40-60', '>60'])

age_results = []
age_groups = df['AgeGroup'].unique()

for group in age_groups:
    if pd.isna(group): continue
    subset = df[df['AgeGroup'] == group]
    if len(subset) == 0: continue
    
    fnr, fpr, tn, fp, fn, tp = calculate_metrics(subset['TrueLabel'], subset['Prediction'])
    age_results.append({
        'Group': group,
        'FNR': fnr,
        'FPR': fpr,
        'Count': len(subset)
    })

age_results_df = pd.DataFrame(age_results).sort_values('Group')
print(age_results_df)

In [ ]:
plt.figure(figsize=(10, 5))
sns.barplot(data=age_results_df, x='Group', y='FNR')
plt.title('Tasa de Falsos Negativos (FNR) por Grupo de Edad')
plt.ylabel('FNR (Menor es mejor)')
plt.show()

## 5. Conclusiones

**Interpretación:**
- Un FNR más alto en un grupo indica que el modelo falla más en detectar la enfermedad en ese grupo, lo cual es crítico en salud.
- Diferencias significativas entre grupos sugieren sesgo algorítmico o de datos.

**Hallazgos:**
- (Escribir aquí los hallazgos observados tras la ejecución)